In [1]:
# Matplotlib
import matplotlib.pyplot as plt
# Tensorflow
import tensorflow as tf

# from tensorflow import keras as keras

# import pytorch as torch
# Numpy and Pandas
import numpy as np
import pandas as pd
# Ohter import
import sys

In [4]:
assert hasattr(tf, "function")  # Be sure to use tensorflow 2.0

## Create cutomized layers

In [49]:
# Not useful in code but it is useful to understand the concept

class CustomLayers(tf.keras.layers.Layer):

    def __init__(self, units, activations):
        super(CustomLayers, self).__init__()
        self.units = units
        self.activations_list = activations
        # self.num_outputs = num_outputs
        self.weights_list = []

    # The build function gets called the first time your layer is used. 
    # Creating variables on build() allows you to make their shape depend on the input shape and hence removes the need for the user to specify full shapes. 
    # It is possible to create variables during __init__() if you already know their full shapes.
    def build(self, input_shape):

        # print('input_shape', input_shape)

        # enumerate each layer
        # i == the index of the layers
        # unit == the number of neurons in the layer
        for i, unit in enumerate(self.units):
            print('i', i)
            print('unit', unit)
            print('input_shape', input_shape)
            # self.add_weight(name='kernel'+str(i), shape=[int(input_shape[-1]), unit])
            # input_shape = unit

            weights = self.add_weight(
                name='weights'+str(i), 
                shape=[int(input_shape[1]), unit],
                initializer='uniform', # random_uniform # random_normal
                trainable=True)
            
            self.weights_list.append(weights)

            input_shape = (None, unit)

        super(CustomLayers, self).build(input_shape)

        # self.kernel = self.add_variable("kernel", shape=[int(input_shape[-1]), self.num_outputs])

    def call(self, input):

        # print('Call function')

        # Computations of the forward propagation
        for weights, activation in zip(self.weights_list, self.activations_list):
            # print('weights', weights)
            # print('activation', activation)

            # dot product
            input = tf.matmul(input, weights) # + bias 

            # activation
            if activation == "relu":
                input = tf.nn.relu(input)
            elif activation == "softmax":
                input = tf.nn.softmax(input)
            elif activation == "sigmoid":
                input = tf.nn.sigmoid(input)

        return input


model = tf.keras.Sequential()

print("Add layers")
model.add(tf.keras.layers.Dense(4))
model.add(CustomLayers([4, 2], ["relu", "softmax"]))

print("Make a prediction")
model.predict(np.zeros([1, 6]))
# model.predict(np.zeros([1, 10]))

Add layers
Make a prediction
i 0
unit 4
input_shape (None, 4)
i 1
unit 2
input_shape (None, 4)
1/1 [==============================] - 0s 198ms/step


array([[0.5, 0.5]], dtype=float32)

## Create a model using the SubClassing API

In [24]:
class CustomModel(tf.keras.Model):

    def __init__(self):
        super(CustomModel, self).__init__()

        # self.dense1 = tf.keras.layers.Dense(4, activation=tf.nn.relu)
        # self.dense2 = tf.keras.layers.Dense(5, activation=tf.nn.softmax)

        self.first_layer = tf.keras.layers.Dense(64, activation=tf.nn.relu, input_shape=(784,))
        self.output_layer = tf.keras.layers.Dense(10, activation=tf.nn.softmax)

    def call(self, inputs):

        # In the call method, you can do tf. operations on TensorFlow tensors.

        layer_1 = self.first_layer(inputs)
        output = self.output_layer(layer_1)
        return output
    

try:
    model = CustomModel()
except Exception as e:
    print("Error : ", e)
    sys.exit(1)

In [25]:
model.predict(np.zeros((1, 784)))

1/1 [==============================] - 0s 162ms/step


array([[0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]], dtype=float32)

In [26]:
model.compile(
    loss=tf.keras.losses.sparse_categorical_crossentropy,
    optimizer=tf.keras.optimizers.Adam(0.001),
    metrics=["accuracy"])

## Mnist problem

In [27]:
# Dataset

from sklearn.preprocessing import StandardScaler
# Fashio MNIST
fashion_mnist = tf.keras.datasets.fashion_mnist
(images, targets), (_, _) = fashion_mnist.load_data()
# Get only a subpart of the dataset
images = images[:10000]
targets = targets [:10000]
# Reshape the dataset and convert to float
images = images.reshape(-1, 784)
images = images.astype(float)
# Normalize images
scaler = StandardScaler()
images = scaler.fit_transform(images)

print("images.shape : ", images.shape)
print("targets.shape : ", targets.shape)



images.shape :  (10000, 784)
targets.shape :  (10000,)


In [28]:
model.predict(images[0:1])

1/1 [==============================] - 0s 115ms/step


array([[0.343528  , 0.00743907, 0.01926639, 0.17100503, 0.03084405,
        0.02232789, 0.02764364, 0.07821842, 0.18341577, 0.1163118 ]],
      dtype=float32)

In [29]:
history = model.fit(images, targets, epochs=10, validation_split=0.2)

Epoch 1/10


250/250 [==============================] - 2s 5ms/step - loss: 0.6478 - accuracy: 0.7731 - val_loss: 0.4657 - val_accuracy: 0.8335
Epoch 2/10
250/250 [==============================] - 1s 6ms/step - loss: 0.4131 - accuracy: 0.8497 - val_loss: 0.4611 - val_accuracy: 0.8435
Epoch 3/10
250/250 [==============================] - 2s 9ms/step - loss: 0.3386 - accuracy: 0.8781 - val_loss: 0.4387 - val_accuracy: 0.8420
Epoch 4/10
250/250 [==============================] - 3s 14ms/step - loss: 0.3048 - accuracy: 0.8892 - val_loss: 0.4643 - val_accuracy: 0.8375
Epoch 5/10
250/250 [==============================] - 2s 6ms/step - loss: 0.2656 - accuracy: 0.9051 - val_loss: 0.4143 - val_accuracy: 0.8545
Epoch 6/10
250/250 [==============================] - 3s 12ms/step - loss: 0.2324 - accuracy: 0.9165 - val_loss: 0.4631 - val_accuracy: 0.8500
Epoch 7/10
250/250 [==============================] - 1s 5ms/step - loss: 0.2083 - accuracy: 0.9255 - val_loss: 0.4524 - val_accuracy: 0.8425
Ep